<a href="https://colab.research.google.com/github/Tejaswi4563/AI-Agent/blob/main/front_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Step 1: Install required libraries
# This cell installs Gradio and Requests, which are necessary for the app to run.
# It uses the '!' prefix to execute shell commands within Google Colab.
!pip install gradio requests

# @title Step 2: Import libraries
# Import necessary modules from gradio for building the UI,
# requests for making HTTP calls to the NewsAPI,
# and datetime/timedelta for handling dates to fetch news from the last 7 days.
import gradio as gr
import requests
from datetime import datetime, timedelta

# @title Step 3: News fetch logic
# This function interacts with the NewsAPI to retrieve political news.
# It now takes a city and country name as input, and the API key is hardcoded.
def get_political_news(city_name, country_name):
    # !!! IMPORTANT: Replace "YOUR_NEWSAPI_KEY_HERE" with your actual NewsAPI key !!!
    # This is hardcoded for demonstration as per your request, but NOT recommended for production.
    api_key = "fa7e205f8c444fe5b5fedf39f4636a86" # <--- REPLACE THIS WITH YOUR REAL API KEY

    url = "https://newsapi.org/v2/everything"

    # Define a list of keywords to filter for political news.
    political_keywords = [
        "politics", "government", "election", "parliament", "minister",
        "president", "policy", "congress", "diplomacy", "political party"
    ]

    # Validate if at least one location input is provided.
    if not city_name and not country_name:
        return "Please enter either a city name or a country name to search for news."

    # Construct the location part of the query
    location_query_parts = []
    if city_name:
        location_query_parts.append(city_name)
    if country_name:
        location_query_parts.append(country_name)

    location_query = " AND ".join(location_query_parts)

    # Construct the full query: location AND political keywords
    query = f"({location_query}) AND ({' OR '.join(political_keywords)})"

    # Calculate the date 7 days ago to fetch recent news.
    from_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")

    # Set up parameters for the NewsAPI request.
    params = {
        "q": query,            # The search query
        "from": from_date,     # Start date for the news search
        "language": "en",      # Only English articles
        "sortBy": "publishedAt", # Sort by publication date
        "pageSize": 10,        # Get up to 10 articles
        "apiKey": api_key      # Your NewsAPI key (now hardcoded)
    }

    try:
        # Make the GET request to the NewsAPI.
        response = requests.get(url, params=params)

        # Check if the API request was successful (status code 200).
        if response.status_code != 200:
            # If not successful, extract and return the error message from the API.
            error_message = response.json().get('message', 'Unknown error occurred with the API.')
            return f"❌ Error from NewsAPI: {error_message}"

        data = response.json()
        articles = data.get("articles", []) # Get the list of articles, default to empty list

        # If no articles are found, inform the user.
        if not articles:
            return f"⚠️ No political news found for '{city_name} {country_name}' in the past week. Try different inputs or broaden your search."

        # Format the fetched articles into a readable string for the Gradio output.
        output = ""
        for i, article in enumerate(articles, 1):
            title = article.get("title", "No Title Available") # Safely get title
            desc = article.get("description", "") or "No description available." # Safely get description, handle None
            url = article.get("url", "#") # Safely get URL
            date = article.get("publishedAt", "")[:10] # Safely get and format date

            output += f"**{i}. [{title}]({url})**\n" \
                      f"📅 {date}\n" \
                      f"📝 {desc}\n\n"
        return output

    except requests.exceptions.RequestException as req_e:
        # Catch network-related errors (e.g., no internet connection).
        return f"❌ Network or connection error: Please check your internet connection. ({str(req_e)})"
    except Exception as e:
        # Catch any other unexpected errors.
        return f"❌ An unexpected error occurred: {str(e)}"

# @title Step 4: Gradio interface
# Define the Gradio interface using gr.Interface.
# - fn: The Python function to call when the user interacts with the UI.
# - inputs: A list of Gradio components for user input.
#   - Textbox for City Name.
#   - Textbox for Country Name.
# - outputs: A Gradio component to display the output (Markdown for formatted text).
# - title & description: Text to display at the top of the Gradio app.
# - allow_flagging="never": Disables the "Flag" button.
interface = gr.Interface(
    fn=get_political_news,
    inputs=[
        gr.Textbox(label="🏙️ Enter City Name (Optional)", placeholder="e.g., New Delhi, London"),
        gr.Textbox(label="🌍 Enter Country Name (Optional)", placeholder="e.g., India, United Kingdom")
    ],
    outputs=gr.Markdown(),
    title="🗳️ Political News AI Agent",
    description="Enter a city and/or country to get political news headlines from the last 7 days.",
    allow_flagging="never"
)

# @title Step 5: Launch the app
# Launch the Gradio interface.
# Setting 'share=True' generates a public, temporary URL that you can share.
interface.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6185f36a9f2d61609.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
